#### LUVOIR science case: Spectroscopy of outflows from star-forming regions in distant galaxies.  jrigby, 14 June 2016

In [89]:
%pylab
%matplotlib inline
from astropy.modeling.functional_models import Gaussian2D
from astropy.utils.data import download_file
from astropy.io import fits
import pyds9
import pyregion
from astropy.wcs import WCS
from astropy.wcs.utils import proj_plane_pixel_scales
from astropy import units
from astropy.cosmology import WMAP9 as cosmo

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [90]:
def Jy2AB(Jy): 
    AB = -2.5 * log10(Jy * 1E-23) - 48.57
    return(AB)

In [91]:
# Let's set some variables
D = 10.0 #telescope diameter, in m
zz = 2.0
restwave = 1334 # in Angstroms.  This is C II, a strong ISM absorption line

In [104]:
# Derived quantities
D_L = cosmo.luminosity_distance(zz)  # units are Mpc
D_L = D_L.to(units.cm).value
print type(D_L)
wave = restwave * (1.+zz) # in Angstroms
diflim = wave * 1E-10 / D * 206265 # in arcsec
pix_scale = diflim / 2. # Temporarily assuming Nyquist-sampling the diffraction-limit
print "Diffraction limit was ", diflim, "arcsec"
print " at a wavelength of", wave, "Angstroms"

<type 'float'>
Diffraction limit was  0.0082547253 arcsec
 at a wavelength of 4002.0 Angstroms


Hmm, this is close to the pixel scale of S1110 source-plane reconstructions (0.003"/pix).  Optionally, could blur slightly with hacked version of candelize_*py. But that's weird b/c mag isn't same in x, y.  Ponder this.

In [93]:
gal_size = 7. # kpc
kpc_per_arcsec = (1./ cosmo.arcsec_per_kpc_proper(1.0)).value  # kpc in 1 arcsec, should be ~8
ang_size = gal_size / kpc_per_arcsec
print "Angular size of galaxy is:\n", ang_size, "arcsec,",
print "which is ", round(ang_size / pix_scale), "pixels across"

Angular size of galaxy is:
0.85865198615 arcsec, which is  208.0 pixels across


This cries out for an IFU! How would that work? Even a 1K x 1K would be amazing.

In [94]:
size_IFU = 1024 # N pix on a side
fov = size_IFU * pix_scale
print "IFU of", size_IFU, "by", size_IFU, "has FOV", fov, "arcsec"

IFU of 1024 by 1024 has FOV 4.2264193536 arcsec


Hmm, so similar FOV to JWST IFU, but much finer pixels.

In [95]:
# How bright would a SF region be?  From Livermore et al. (2015), Figure 8, 
# at z~2 the typical measured SFR/surface area is: 
SB = 3.0 /(np.pi * 1E3**2)  # units of Msol/yr/pc^2
R = 100. # pc, size of a typical region
clump_sfr = SB * np.pi * R**2  # Msol/yr
print clump_sfr, "Msol/yr for a R=", R, "pc clump"
fnu1 = clump_sfr / 1.4E-28 /(4.*np.pi * D_L**2)*(1+zz)/1E-23  #(Jy) eqn 1 of Kennicutt 1998, w bandwidth compression
print "fnu1 = ", fnu1, "Jy, AB1=", Jy2AB(fnu1)

0.03 Msol/yr for a R= 100.0 pc clump
fnu1 =  2.13189388486e-09 Jy, AB1= 30.6080860403



Get simulated z=2.03 galaxy from Greg Snynder's simulations w 12m LUVOIR.

In [100]:
image_file = "/Users/jrrigby1/WORK/LUVOIR/SFR_sci_case/Snyder_sims/VELA27/VELA27_a0.330_CAMERA0-BROADBAND_SB25_HDST12_b.fits"
#(image, hdr) = fits.getdata(file, header=True) #image is numpy array
ff = fits.open(image_file)
# Greg says units are "Fluxes are in observed nanojanskies/Sr, at least according to the headers I saved with them a year or two ago.  "
flux_const = 2.35044E-11 * sim_pixscale**2 * 1E-9  # convert Greg's units to Jy/pix
sim_pixscale = ff[0].header['pixscale']  # "/pix
print sim_pixscale
image = ff[0].data  # screwing around to get hdu as pyregions wants it
image += 0.0 # image is a np ndarray, so you can do math on it
d = pyds9.DS9('foo1')  # start ds9.  'd' is now the way to call ds9
d.set_np2arr(image)
d.set("colorbar no")   
d.set("scale zscale")  

0.006


1

In [97]:
# load a regions file
reg_name = "/Users/jrrigby1/WORK/LUVOIR/SFR_sci_case/Snyder_sims/test4.reg"
bkg_reg =  "/Users/jrrigby1/WORK/LUVOIR/SFR_sci_case/Snyder_sims/blank.reg"
all_reg = "/Users/jrrigby1/WORK/LUVOIR/SFR_sci_case/Snyder_sims/all.reg"
d.set('regions load', reg_name)  # load that region in ds9

b = pyregion.open(bkg_reg).as_imagecoord(header=ff[0].header) 
bkg_level = np.median(b.get_mask(ff[0]) * image)   # median bkg in image

a = pyregion.open(all_reg).as_imagecoord(header=ff[0].header)  # big aperture, all flux
f_tot = (np.sum(a.get_mask(ff[0])*image) -  bkg_level * np.sum(a.get_mask(ff[0])))*flux_const

r = pyregion.open(reg_name).as_imagecoord(header=ff[0].header)  # small aperture
f_clump = (np.sum(r.get_mask(ff[0])*image) -  bkg_level * np.sum(r.get_mask(ff[0])))*flux_const

print f_tot, Jy2AB(f_tot), f_clump, Jy2AB(f_clump)

8.8730586546e-07 24.059816619 4.69319651893e-09 29.7513281503


OK, so if I understood Greg's units, his example galaxy has an m_AB(B)=24.0, and a given R=400pc clump is about m_AB(B)=29.7. More or less consistent w Livermore scaling above, since Livermore picked brightest regions. 

JT's **imaging** ETC says: 
u-band: D=12m, t=1hr, m_AB=29 gives S/N=34
B-band: D=12m, t=1hr, m_AB=29.7 gives S/N=37.
So, we can certainly image such lumps, but we can't get good spectra.
What about bigger slices of the galaxy?  (20 slices would each be down 3.2 mags from the total, ~27.3 each)

  Also, out to what redshift in the nearby universe could one do Crowther 2016 paper?  